## POS tagging using modified Viterbi

### Data Preparation

In [4]:
#Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time

In [5]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [6]:
#Splitting the datset into training and test sets. Training =95% Testing=5%
train_set,test_set = train_test_split(nltk_data,train_size=0.95,test_size=0.05,random_state = 101)

In [7]:
#List of train and test tagged words
train_tagged_words = [w for word in train_set for w in word]
test_tagged_words = [w[0] for word in test_set for w in word]
print(len(train_tagged_words))
print(len(test_tagged_words))

95547
5129


In [8]:
#Printing few words in train set
print(train_tagged_words[1:5])

[('confirmed', 'VERB'), ('the', 'DET'), ('filing', 'NOUN'), ('but', 'CONJ')]


In [9]:
#Printing few words in test set
print(test_tagged_words[1:5])

['company', 'said', '0', 'it']


In [10]:
#Length of Unique tags and vocabulary check
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
vocab = {word for word,tag in train_tagged_words}
print(len(vocab))

12
12100


Now we have completed the basic steps and basis checks needed in order to perform the POS tagging so lets use Viterbi Algorithm for POS tagging

### Build the vanilla Viterbi based POS tagger

#### Vanilla Viterbi POS tagger 

In [11]:
## Emission Probablity
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

In [12]:
## Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [13]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [14]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))

In [15]:
tags_df

,VERB,CONJ,NUM,PRT,NOUN,.,ADV,DET,ADJ,X,PRON,ADP
VERB,0.169249,0.005577,0.022851,0.030674,0.110070,0.034934,0.081952,0.134392,0.064988,0.217506,0.035786,0.092022
CONJ,0.156671,0.000465,0.039981,0.004649,0.349140,0.034868,0.055323,0.121339,0.118085,0.008833,0.058113,0.052534
NUM,0.018761,0.013699,0.184932,0.026504,0.350208,0.117332,0.002978,0.003276,0.034247,0.210542,0.001489,0.036033
PRT,0.405272,0.002306,0.056672,0.001647,0.247776,0.043822,0.010214,0.097858,0.083031,0.013509,0.017792,0.020099
NOUN,0.147667,0.042666,0.009542,0.043397,0.263564,0.240604,0.017074,0.012942,0.012248,0.029175,0.004607,0.176514
.,0.089095,0.057538,0.081003,0.002427,0.222242,0.093320,0.052324,0.173335,0.043963,0.026971,0.066349,0.091342
ADV,0.343491,0.006956,0.030474,0.014243,0.031467,0.137131,0.080490,0.069891,0.129182,0.023186,0.014906,0.118582
DET,0.039850,0.000483,0.022220,0.000242,0.638087,0.017993,0.012438,0.005676,0.204323,0.045405,0.003744,0.009540
ADJ,0.011699,0.016971,0.021256,0.010710,0.699621,0.063931,0.004778,0.004943,0.066403,0.021091,0.000330,0.078267
X,0.203851,0.010662,0.002864,0.185232,0.062381,0.163590,0.024984,0.054742,0.017187,0.076384,0.055538,0.142584


In [16]:
#Vanilla Viterbi Heuristic

def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [17]:
# choose the random seed value as 1234
random.seed(1234)

# choose random 5 sentences
rndom = [random.randint(1,len(test_set)) for x in range(5)]

# list of sentences
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
test_run

[[('The', 'DET'),
  ('Contra', 'NOUN'),
  ('military', 'ADJ'),
  ('command', 'NOUN'),
  (',', '.'),
  ('in', 'ADP'),
  ('a', 'DET'),
  ('statement', 'NOUN'),
  ('from', 'ADP'),
  ('Honduras', 'NOUN'),
  (',', '.'),
  ('said', 'VERB'),
  ('0', 'X'),
  ('Sandinista', 'NOUN'),
  ('troops', 'NOUN'),
  ('had', 'VERB'),
  ('launched', 'VERB'),
  ('a', 'DET'),
  ('major', 'ADJ'),
  ('offensive', 'NOUN'),
  ('against', 'ADP'),
  ('the', 'DET'),
  ('rebel', 'NOUN'),
  ('forces', 'NOUN'),
  ('.', '.')],
 [('*-1', 'X'),
  ('Bucking', 'VERB'),
  ('the', 'DET'),
  ('market', 'NOUN'),
  ('trend', 'NOUN'),
  (',', '.'),
  ('an', 'DET'),
  ('issue', 'NOUN'),
  ('of', 'ADP'),
  ('$', '.'),
  ('130', 'NUM'),
  ('million', 'NUM'),
  ('*U*', 'X'),
  ('general', 'ADJ'),
  ('obligation', 'NOUN'),
  ('distributable', 'ADJ'),
  ('state', 'NOUN'),
  ('aid', 'NOUN'),
  ('bonds', 'NOUN'),
  ('from', 'ADP'),
  ('Detroit', 'NOUN'),
  (',', '.'),
  ('Mich.', 'NOUN'),
  (',', '.'),
  ('apparently', 'ADV'),
  ('drew'

In [18]:
# Time taken
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
print("Time taken is: ",difference)

Time taken is:  34.879995822906494


In [19]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = (len(check)/len(tagged_seq))*100
print("Accuracy obtained is: ",accuracy)

Accuracy obtained is:  91.1504424778761


In [20]:
incorrect_tagged_cases = [j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]

In [21]:
incorrect_tagged_cases

[(('Contra', 'VERB'), ('Contra', 'NOUN')),
 (('command', 'VERB'), ('command', 'NOUN')),
 (('Honduras', 'VERB'), ('Honduras', 'NOUN')),
 (('Sandinista', 'VERB'), ('Sandinista', 'NOUN')),
 (('offensive', 'VERB'), ('offensive', 'NOUN')),
 (('rebel', 'VERB'), ('rebel', 'NOUN')),
 (('forces', 'VERB'), ('forces', 'NOUN')),
 (('Eveready', 'VERB'), ('Eveready', 'NOUN')),
 (('*T*-252', 'VERB'), ('*T*-252', 'X')),
 (('up', 'ADV'), ('up', 'PRT'))]

The above words are incorrectly tagged with Vanilla Viterbi algorithm

### Solve the problem of unknown words

### Modified Viterbi Algorithm - 1

In Modified Viterbi Algorithm - 1, Vanilla Viterbi is modified such that if unknown word is present, we only use transition probability when emission probability is 0.

In [22]:
# In the case of unknown words, use transition probability when emission probability is zero.

def Viterbi_modified_1(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        p_transition =[] # storing transition probabilities
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        
      
        # If Probability == zero (occurence of unknown word) -> transition probability is used
        if(pmax==0):
            pmax = max(p_transition)
            state_max = T[p_transition.index(pmax)]
                           
        else:
            state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))

In [23]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_modified_1(test_tagged_words)
end = time.time()
difference = end-start
print("Time taken is: ",difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print("Accuracy obtained is: ",accuracy*100)

Time taken is:  28.946441411972046
Accuracy obtained is:  94.69026548672566


In [24]:
# Incorrectly tagged words after Technique-1
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('command', 'VERB'), ('command', 'NOUN')),
 (('Honduras', 'DET'), ('Honduras', 'NOUN')),
 (('Sandinista', 'VERB'), ('Sandinista', 'NOUN')),
 (('Eveready', 'VERB'), ('Eveready', 'NOUN')),
 (('*T*-252', 'VERB'), ('*T*-252', 'X')),
 (('up', 'ADV'), ('up', 'PRT'))]

Compared to Vanilla Viterbi, Modified Viterbi 1 tagged few words correctly such as: Contra, rebel, offensive and forces.

### Modified Viterbi Algorithm - 2

In this algorithm, the unknown words are identified using rule based tagging

In [25]:
# Patterns used for tagging
pattern = [
    (r'.*ing$', 'VERB'),              
    (r'.*ed$', 'VERB'),                
    (r'.*es$', 'VERB'),                  
    (r'.*\'s$', 'NOUN'),              
    (r'.*s$', 'NOUN'),                
    (r'\*T?\*?-[0-9]+$', 'X'),        
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), 
    (r'.*', 'NOUN')                   
]

# rule based tagger
rule_based_tagger = nltk.RegexpTagger(pattern)

In [26]:
# Rule based tagger is used in case of occurence of an unknown word
def Viterbi_modified_2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1]       
       
        
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] 
        else:
            if state_max != 'X':
                state_max = T[p.index(pmax)]                
            
        
        state.append(state_max)
    return list(zip(words, state))

In [27]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_modified_2(test_tagged_words)
end = time.time()
difference = end-start
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)

print("Time taken is: ",difference)
print("Accuracy obtained is: ",accuracy*100)

Time taken is:  28.451680183410645
Accuracy obtained is:  97.34513274336283


In [28]:
# Incorrectly tagged words after Technique-2
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('command', 'VERB'), ('command', 'NOUN')),
 (('drew', 'NOUN'), ('drew', 'VERB')),
 (('up', 'ADV'), ('up', 'PRT'))]

Compared to Vanilla Viterbi, Modified Viterbi 2 tagged few words correctly such as: Contra, Honduras, Sandinista, offensive, rebel, forces, *T*-252, and eveready.

Compared to Modified Viterbi 1, Modified Viterbi 2 tagged few words correctly such as: Honduras, Sandinista, T-252 and eveready.

#### Evaluating tagging accuracy

In [29]:
# list of tagged words
test_run_base_full = [tup for sent in test_set for tup in sent]

# list of untagged words
test_tagged_words_full = [tup[0] for sent in test_set for tup in sent]

#### Vanilla Viterbi Technique

In [30]:
start = time.time()
tagged_seq = Viterbi(test_tagged_words_full)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)

Time taken in seconds:  1272.6303236484528


In [31]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base_full) if i == j] 
accuracy = len(check)/len(tagged_seq)
print("Accuracy obtained is: ",accuracy*100)

Accuracy obtained is:  92.12322090076039


#### Modified Viterbi technique 1 

In [32]:
start = time.time()
tagged_seq = Viterbi_modified_1(test_tagged_words_full)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)

Time taken in seconds:  1233.4011788368225


In [33]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base_full) if i == j] 
accuracy = len(check)/len(tagged_seq)
print("Accuracy obtained is: ",accuracy*100)

Accuracy obtained is:  94.03392474166505


#### Modified Viterbi technique 2

In [34]:
start = time.time()
tagged_seq = Viterbi_modified_2(test_tagged_words_full)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)

Time taken in seconds:  1207.428570985794


In [35]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base_full) if i == j] 
accuracy = len(check)/len(tagged_seq)
print("Accuracy obtained is: ",accuracy*100)

Accuracy obtained is:  95.37921622148568


#### POS using Viterbi and modified Viterbi on Sample test cases given

In [36]:
Test_sentences = ["Android is a mobile operating system developed by Google.",
"Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.",
"Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.",
"Twitter is an online news and social networking service on which users post and interact with messages known as tweets.",
"Before entering politics, Donald Trump was a domineering businessman and a television personality.",
"The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.",
"This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.",
"Show me the cheapest round trips from Dallas to Atlanta",
"I would like to see flights from Denver to Philadelphia.",
"Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.",
"NASA invited social media users to experience the launch of ICESAT-2 Satellite."]

In [37]:
sample_test_words = [w for sent in Test_sentences for w in sent.split()]

#### Vanilla Viterbi on sample test cases

In [38]:
# tagging the test sentences
start = time.time()
sample_tagged_seq = Viterbi(sample_test_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  38.2150297164917


In [39]:
sample_tagged_seq

[('Android', 'VERB'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google.', 'VERB'),
 ('Android', 'VERB'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'VERB'),
 ('worldwide', 'VERB'),
 ('on', 'ADP'),
 ('smartphones', 'VERB'),
 ('since', 'ADP'),
 ('2011', 'VERB'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013.', 'VERB'),
 ('Google', 'VERB'),
 ('and', 'CONJ'),
 ('Twitter', 'VERB'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'VERB'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'VERB'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ("Twitter's", 'VERB'),
 ('firehose.', 'VERB'),
 ('Twitter', 'VERB'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'VERB'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('service', 'NOUN'),
 ('on', 'ADP'),
 ('which', 'D

#### Modified Viterbi technique 1 on sample test cases

In [40]:
# tagging the test sentences
start = time.time()
sample_tagged_seq2 = Viterbi_modified_1(sample_test_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  39.75897264480591


In [41]:
sample_tagged_seq2

[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google.', 'DET'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'DET'),
 ('since', 'ADP'),
 ('2011', 'DET'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013.', 'DET'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'DET'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'X'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ("Twitter's", 'VERB'),
 ('firehose.', 'X'),
 ('Twitter', 'VERB'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('service', 'NOUN'),
 ('on', 'ADP'),
 ('which', 'DET'),
 ('us

#### Modified Viterbi technique 2 on sample test cases

In [42]:
# tagging the test sentences
start = time.time()
sample_tagged_seq3 = Viterbi_modified_2(sample_test_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  42.17041516304016


In [43]:
sample_tagged_seq3

[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google.', 'NOUN'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'VERB'),
 ('since', 'ADP'),
 ('2011', 'NUM'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013.', 'NOUN'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'NUM'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ("Twitter's", 'NOUN'),
 ('firehose.', 'NOUN'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('service', 'NOUN'),
 ('on', 'ADP'),
 ('which', 'DET

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

#### Accuracy of Vanilla Viterbi = 92.12%
#### Accuracy of Modified Viterbi technique-1 = 94.03%
In this technique, transition probability alone is considered for unknown words when emission probability is zero.  
#### Accuracy of Modified Viterbi technique-2 = 95.38%
In this technique, Rule based taggers are used in cases of unknown words
Amongst the modified techniques, rule based taggers used in cases of unknown words has highest accuracy of about 95.4% compared to Vanilla Viterbi having accuracy of about 92%

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

##### Words that got incorrectly tagged as a part of Vanilla Viterbi
Contra
<br>command
<br>Honduras
<br>Sandinista 
<br>Eveready
<br>T-252 
<br>rebel
<br>forces
<br>up
<br>offensive

##### Below are the words that got correctly tagged after using modified tecnique -1 as compared to Vanilla Viterbi:
Contra as Noun,
offensive as Noun,
rebel as Noun and forces as Noun

##### Below are the words that got correctly tagged after using modified tecnique -2 as compared to Vanilla Viterbi:
Contra as Noun, Honduras as Noun, Sandinista as Noun, offensive as Noun, rebel as Noun, forces as Noun, *T*-252 as X and eveready as Noun.

##### Below are the words that got correctly tagged after using modified tecnique -2 as compared to modified technique -1:
Honduras as Noun, Sandinista as Noun, T-252 as X and eveready as Noun.